Trabalho Feito por:

Eugênia Cornils

Tiago Leite

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=2fd31b58928c22e8938d15c08b48b5a1304f4411a3d2c8b9591b052db8252756
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
!java -version


openjdk version "11.0.24" 2024-07-16
OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu322.04)
OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu322.04, mixed mode, sharing)


In [ ]:
!echo $JAVA_HOME
!echo $SPARK_HOME


In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Adicionar um índice sequencial para cada sequência
windowSpec = Window.orderBy("index")
data_df = data_df.withColumn("index", row_number().over(windowSpec) - 1)


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `index` cannot be resolved. Did you mean one of the following? [`col_0`, `col_1`, `col_2`, `col_3`, `col_4`].;
'Project [col_0#0L, col_1#1L, col_2#2L, col_3#3L, col_4#4L, col_5#5L, col_6#6L, col_7#7L, col_8#8L, col_9#9L, col_10#10L, col_11#11L, col_12#12L, col_13#13L, col_14#14L, col_15#15L, col_16#16L, col_17#17L, col_18#18L, col_19#19L, col_20#20L, col_21#21L, col_22#22L, col_23#23L, ... 143 more fields]
+- LogicalRDD [col_0#0L, col_1#1L, col_2#2L, col_3#3L, col_4#4L, col_5#5L, col_6#6L, col_7#7L, col_8#8L, col_9#9L, col_10#10L, col_11#11L, col_12#12L, col_13#13L, col_14#14L, col_15#15L, col_16#16L, col_17#17L, col_18#18L, col_19#19L, col_20#20L, col_21#21L, col_22#22L, col_23#23L, ... 142 more fields], false


In [ ]:
from pyspark.sql.types import IntegerType

# Aplicar o mapeamento para que os índices estejam no intervalo correto
pairs_df = pairs_df.withColumn("index1", udf(lambda id: index_mapping[id], IntegerType())(col("index1")))
pairs_df = pairs_df.withColumn("index2", udf(lambda id: index_mapping[id], IntegerType())(col("index2")))


In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import numpy as np
from pyspark.sql.functions import col, array, monotonically_increasing_id, udf
from pyspark.sql.types import FloatType


In [ ]:
# Configuração do Spark
conf = SparkConf() \
    .setAppName("ButinaClustering") \
    .setMaster("local[*]") \
    .set("spark.executor.memory", "4g") \
    .set("spark.driver.memory", "4g") \
    .set("spark.driver.maxResultSize", "2g") \
    .set("spark.sql.shuffle.partitions", "200")

spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [ ]:
# Gerar 50 sequências binárias de 50 elementos
num_sequences = 1000
sequence_length = 166
np.random.seed(42)
data = np.random.randint(2, size=(num_sequences, sequence_length))

# Converter dados para DataFrame
columns = [f'col_{i}' for i in range(sequence_length)]
data_df = spark.createDataFrame(data.tolist(), columns)

# Adicionar um índice sequencial para cada sequência
window = Window.orderBy(monotonically_increasing_id())
data_df = data_df.withColumn("index", row_number().over(window) - 1)

In [ ]:
# UDF para calcular similaridade
def calculate_similarity_udf(seq1, seq2):
    return float(1 - np.mean(np.array(seq1) != np.array(seq2)))

In [ ]:
similarity_udf = udf(calculate_similarity_udf, FloatType())

# Criar um DataFrame para pares de sequências e calcular a similaridade
data_df1 = data_df.alias("df1")
data_df2 = data_df.alias("df2")
pairs_df = data_df1.crossJoin(data_df2) \
    .filter(col("df1.index") < col("df2.index"))

pairs_df = pairs_df.withColumn("similarity", similarity_udf(
    array([col(f'df1.col_{i}') for i in range(sequence_length)]),
    array([col(f'df2.col_{i}') for i in range(sequence_length)])
))

# Coletar resultados e formar a matriz de similaridade
similarity_matrix = np.zeros((num_sequences, num_sequences))

In [ ]:
for row in pairs_df.select(col("df1.index").alias("index1"), col("df2.index").alias("index2"), "similarity").collect():
    i, j, similarity = row
    similarity_matrix[i, j] = similarity
    similarity_matrix[j, i] = similarity


In [ ]:
# Função do algoritmo de Butina
def butina_clustering(similarity_matrix, d_cut):
    num_objects = similarity_matrix.shape[0]
    ungrouped = list(range(num_objects))
    clusters = []

    # Calcular o número de vizinhos para cada objeto dentro do limiar d_cut
    neighbor_counts = np.sum(similarity_matrix >= d_cut, axis=1)

    # Ordenar os objetos com base no número de vizinhos em ordem decrescente
    sorted_indices = np.argsort(-neighbor_counts)

    while ungrouped:
        # Selecionar a semente do cluster (objeto com mais vizinhos)
        seed_index = sorted_indices[np.isin(sorted_indices, ungrouped)][0]
        cluster = [seed_index]

        # Encontrar todos os vizinhos da semente dentro do limiar d_cut
        for obj_index in ungrouped:
            if similarity_matrix[seed_index, obj_index] >= d_cut:
                cluster.append(obj_index)

        # Adicionar o cluster à lista de clusters
        clusters.append(cluster)

        # Remover os objetos agrupados da lista de não agrupados
        ungrouped = [obj for obj in ungrouped if obj not in cluster]

    return clusters

In [ ]:
# Definir o limiar de similaridade
d_cut = 0.7

# Executar o algoritmo de Butina
clusters = butina_clustering(similarity_matrix, d_cut)

# Exibir os clusters formados
for i, cluster in enumerate(clusters):
    print(f"Cluster {i+1}: {cluster}")


Cluster 1: [0]
Cluster 2: [658]
Cluster 3: [659]
Cluster 4: [660]
Cluster 5: [661]
Cluster 6: [662]
Cluster 7: [663]
Cluster 8: [664]
Cluster 9: [665]
Cluster 10: [666]
Cluster 11: [667]
Cluster 12: [668]
Cluster 13: [669]
Cluster 14: [670]
Cluster 15: [671]
Cluster 16: [672]
Cluster 17: [673]
Cluster 18: [674]
Cluster 19: [675]
Cluster 20: [676]
Cluster 21: [677]
Cluster 22: [678]
Cluster 23: [679]
Cluster 24: [680]
Cluster 25: [681]
Cluster 26: [682]
Cluster 27: [683]
Cluster 28: [684]
Cluster 29: [657]
Cluster 30: [685]
Cluster 31: [656]
Cluster 32: [654]
Cluster 33: [627]
Cluster 34: [628]
Cluster 35: [629]
Cluster 36: [630]
Cluster 37: [631]
Cluster 38: [632]
Cluster 39: [633]
Cluster 40: [634]
Cluster 41: [635]
Cluster 42: [636]
Cluster 43: [637]
Cluster 44: [638]
Cluster 45: [639]
Cluster 46: [640]
Cluster 47: [641]
Cluster 48: [642]
Cluster 49: [643]
Cluster 50: [644]
Cluster 51: [645]
Cluster 52: [646]
Cluster 53: [647]
Cluster 54: [648]
Cluster 55: [649]
Cluster 56: [650]
Clu